<a href="https://colab.research.google.com/github/Rahafsjob/Capstone_Project_WeCloudData/blob/main/Stage2/DataGovernance_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pip install SQLAlchemy

In [ ]:
import pymysql

# Connect to your RDS MySQL instance
connection = pymysql.connect(
    host='database-1.cipmeg2oi9nm.us-east-1.rds.amazonaws.com',  # ← replace with your RDS host
    user='admin',                            # ← replace with your username
    password='Rahaf123_',                # ← replace with your password
    database='demo_db',                # ← replace with your DB name
    port=3306
)

print("✅ Connected to RDS MySQL")

✅ Connected to RDS MySQL


In [ ]:
# SQL code for email format validation triggers
trigger_code = """
CREATE TRIGGER validate_email_before_insert
BEFORE INSERT ON users
FOR EACH ROW
BEGIN
  IF NEW.email NOT REGEXP '^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,}$' THEN
    SIGNAL SQLSTATE '45000'
    SET MESSAGE_TEXT = '❌ Invalid email format on INSERT!';
  END IF;
END //

CREATE TRIGGER validate_email_before_update
BEFORE UPDATE ON users
FOR EACH ROW
BEGIN
  IF NEW.email NOT REGEXP '^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,}$' THEN
    SIGNAL SQLSTATE '45000'
    SET MESSAGE_TEXT = '❌ Invalid email format on UPDATE!';
  END IF;
END //
"""

In [ ]:
# Execute each SQL statement separately
with connection.cursor() as cursor:
    for statement in trigger_code.split('//'):
        if statement.strip():
            cursor.execute(statement)

connection.commit()
connection.close()
print("✅ Triggers created successfully.")

✅ Triggers created successfully.


In [ ]:
try:
    connection = pymysql.connect(
        host='demo-rds-mysql-db.csb002k2ask1.us-east-1.rds.amazonaws.com',
        user='admin',
        password='Shmuokh001*',
        database='demo_db',
        port=3306
    )

    with connection.cursor() as cursor:
        cursor.execute("""
            INSERT INTO users (id, email, student_id)
            VALUES (9999, 'wrong@@email', 'Stud9999')
        """)
    connection.commit()
    print("❌ This should NOT be printed – trigger failed to block bad email!")

except Exception as e:
    print("✅ Trigger blocked the insert as expected!")
    print("MySQL Error Message:", e)

finally:
    connection.close()

✅ Trigger blocked the insert as expected!
MySQL Error Message: (1644, '❌ Invalid email format on INSERT!')


In [ ]:
try:
    connection = pymysql.connect(
        host='demo-rds-mysql-db.csb002k2ask1.us-east-1.rds.amazonaws.com',
        user='admin',
        password='Shmuokh001*',
        database='demo_db',
        port=3306
    )

    with connection.cursor() as cursor:
        cursor.execute("""
            INSERT INTO users (id, email, student_id)
            VALUES (10000, 'correct.email@example.com', 'Stud10000')
        """)
    connection.commit()
    print("✅ Correct email inserted successfully.")

except Exception as e:
    print("❌ Failed to insert valid email.")
    print("MySQL Error Message:", e)

finally:
    connection.close()

✅ Correct email inserted successfully.
